In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:
# Load the dataset
df = pd.read_csv('diabetes_process.csv')

# Display the first few rows of the dataset
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.599998,0.627,50,-1
1,1,85,66,29,0,26.600000,0.351,31,1
2,8,183,64,0,0,23.299999,0.672,32,-1
3,1,89,66,23,94,28.100000,0.167,21,1
4,0,137,40,35,168,43.099998,2.288,33,-1


In [3]:
# Split the data into features and target
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape


((614, 8), (154, 8))

# Single-layer perceptron

In [4]:
class Perceptron:
    def __init__(self, learning_rate=0.01, n_epochs=1000):
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Initialize weights and bias
        self.weights = np.zeros(n_features)
        self.bias = 0

        # Iterate through the dataset for n_epochs times
        for _ in range(self.n_epochs):
            for idx, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_pred = np.sign(linear_output)
                
                # Update weights and bias
                update = self.learning_rate * (y[idx] - y_pred)
                self.weights += update * x_i
                self.bias += update

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        return np.sign(linear_output)

# Create Perceptron instance
perceptron = Perceptron(learning_rate=0.01, n_epochs=1000)

# Train the perceptron
perceptron.fit(X_train.values, y_train.values)


In [5]:
from sklearn.metrics import accuracy_score

# Predict on the test data
y_pred = perceptron.predict(X_test.values)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
accuracy


0.6623376623376623

# Multi-layer perceptron

In [14]:
# Class for Multi-Layer Perceptron (similar format to the single-layer perceptron provided)
class MultiLayerPerceptron:
    def __init__(self, hidden_layers=(150, 100, 50), learning_rate_init=0.001, max_iter=2000):
        from sklearn.neural_network import MLPClassifier
        self.hidden_layers = hidden_layers
        self.learning_rate_init = learning_rate_init
        self.max_iter = max_iter
        self.clf = MLPClassifier(hidden_layer_sizes=self.hidden_layers, 
                                 learning_rate_init=self.learning_rate_init, 
                                 max_iter=self.max_iter, 
                                 activation='relu', 
                                 solver='adam', 
                                 early_stopping=True, 
                                 validation_fraction=0.1, 
                                 random_state=42)

    def fit(self, X, y):
        # Since neural networks (like MLP) are sensitive to feature scales, we'll internally scale the data
        self.scaler = StandardScaler().fit(X)
        X_scaled = self.scaler.transform(X)
        
        # Train the MLP classifier
        self.clf.fit(X_scaled, y)

    def predict(self, X):
        # Scale the data using the same scaler used for training
        X_scaled = self.scaler.transform(X)
        
        # Predict using the trained MLP classifier
        return self.clf.predict(X_scaled)

# Create MLP instance
mlp_instance = MultiLayerPerceptron()

# Train the MLP
mlp_instance.fit(X_train, y_train)

# Predict on the test data
y_pred_mlp = mlp_instance.predict(X_test)

# Calculate accuracy
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
accuracy_mlp


0.7402597402597403